In [1]:
import openpyxl
import pandas as pd
from natsort import index_natsorted
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [55]:
# wenn match schon einmal gemacht wurde
be = pd.read_excel("iii-merge-test-excel.xlsx", sheet_name="III Drucke 1501-1560", dtype={"digi":"bool"}, true_values=["WAHR"], false_values=["FALSCH"])

# erster match
# be = pd.read_excel("Thomschke_III_Drucke_1501-1560.xlsm", sheet_name="III Drucke 1501-1560", dtype="string")
iii = pd.read_csv("../abzug/iii.csv", dtype="string")

In [41]:
len(be.columns)

99

# neuer Ansatz: merge

In [56]:
df = be.merge(iii, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

# idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
df.loc[df["IDN"].isna(), "IDN"] = df.idn
df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

# wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
df.loc[df["idn"].notna(), "digi"] = True
df.loc[df["idn"].isna(), "digi"] = False
df.fillna('', inplace=True)
df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))

# die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
spaltenliste = list(df.columns)[:-21]
# spaltenliste.insert(0,"digi")
# spaltenliste.insert(4, "bbg")
# spaltenliste.insert(7, "signatur_g")

df.to_excel("iii-merge-test.xlsx", index=False, columns=spaltenliste)

In [50]:
df.columns[:-21]

Index(['digi', 'Lfd Nr.', 'AKZ', 'IDN', 'bbg_x', 'Link zum Portal', 'Signatur',
       'signatur_g_x', 'Provenienzmerkmal', 'Material', 'Format',
       'Öffnungswinkel', 'Einschränkungen', 'Glasplatte ', 'Verpackung',
       'Verpackung austauschen ', 'Schadensklasse',
       'notwendige Reparatur(en) vor der Digitalisierung, notwendige Reparatur(en) vor der Digitalisierung',
       'Bemerkungen', 'Fragen/ Hinweise\nan DBSM', 'Fragen an M. Steinberg',
       'nicht\nam Stand-ort', 'Größe ÜF\n(BxH)', 'Breite\n(nur Ausreißer)',
       'Dicke\n(>12 cm)', '12° Format\n(<15 cm)', 'Einband-\nart',
       'Einband über-\nformt (ganz od. teilweise)',
       'Buch bereits restau-riert',
       'hohler/\nfester Rücken (mit Einlage/\nVergoldung?)',
       'Steh-\nkanten\n(bei Perg.)', 'Leder pudert ab/roter Zerfall (extrem)',
       'Einband stark defor-miert', 'Be-schläge bes. auftra-gend',
       'Buch-schließe steif', 'Buch-block Pa./Perg.', 'saures Füll-material',
       'Register-marken', '

# Excel-Dateien mit openpyxl öffnen

In [57]:
wb = openpyxl.load_workbook("iii-merge-test-excel.xlsx")
wb.remove(wb["III Drucke 1501-1560"])
ws = wb.create_sheet("III Drucke 1501-1560", 0)
df = df.astype("object")
for r in dataframe_to_rows(df[spaltenliste], index=False):
    ws.append(r)
ws.add_table(openpyxl.worksheet.table.Table(displayName="Basistabelle", ref=ws.dimensions))
wb.save("iii-merge-test-excel.xlsx")

# Anleitung zur Bearbeitung der zusammengefügten Listen

Die Tabelle enthält ganz vorn die neue Spalte "Digi". Steht dort WAHR, wird das Werk digitalisiert, steht FALSCH, wird es nicht digitalisiert.

Zunächst muss überprüft werden, ob Frau Thomschke Daten in eine Zeile erfasst hat, in der eigentlich angebundene Werke oder sonstige untergeordnete Werke stehen. Diese müssen dann in die Zeile des übergeordneten Satzes verschoben werden, z. B. Qd.

Alle mit FALSCH markierten Zeilen, die angebundene Werke und keine Informationen von Frau Thomschke haben, können gelöscht werden.

Alle mit FALSCH markierte Zeilen, die keine IDN und keine AKZ enthalten, aber eine Signatur, wurden von BE "gefunden" für diese Bände muss ein Datensatz angelegt werden. Wenn das passiert ist, wird die IDN und AKZ in die Tabelle kopiert.







